<a href="https://colab.research.google.com/github/Code-CloudSG/CS506-Computational-Tools-for-Data-Science/blob/master/Done!_DCM_on_SUPPORT_Dataset_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSM on SUPPORT Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/project_folder/MA/auton-survival

/content/gdrive/My Drive/project_folder/MA/auton-survival


The SUPPORT dataset comes from the Vanderbilt University study
to estimate survival for seriously ill hospitalized adults.
(Refer to http://biostat.mc.vanderbilt.edu/wiki/Main/SupportDesc.
for the original datasource.)

In this notebook, we will apply Deep Survival Machines for survival prediction on the SUPPORT data.

### Load the SUPPORT Dataset

The package includes helper functions to load the dataset.

X represents an np.array of features (covariates),
T is the event/censoring times and,
E is the censoring indicator.

In [ ]:
import sys
sys.path.insert(1, "/content/gdrive/MyDrive/project_folder/MA/auton-survival/auton_survival/")

In [ ]:
from dsm import datasets
x, t, e = datasets.load_dataset('SUPPORT')

ModuleNotFoundError: ignored

In [ ]:
import auton_survival


ModuleNotFoundError: ignored

In [ ]:
from auton_survival.models.dsm import DeepSurvivalMachines

In [ ]:
from auton_survival.models.dsm import datasets
x, t, e = datasets.load_dataset('SUPPORT')

In [ ]:
!apt update
!apt install -y cmake

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,533 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,937 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricte

In [ ]:
!pip install scikit-survival

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 33.0 MB/s 


In [ ]:
!pwd

/content/gdrive/MyDrive/project_folder/MA/auton-survival


### Compute horizons at which we evaluate the performance of DSM

Survival predictions are issued at certain time horizons. Here we will evaluate the performance
of DSM to issue predictions at the 25th, 50th and 75th event time quantile as is standard practice in Survival Analysis.

In [ ]:
import numpy as np
horizons = [0.25, 0.5, 0.75]
times = np.quantile(t[e==1], horizons).tolist()

### Splitting the data into train, test and validation sets

We will train DSM on 70% of the Data, use a Validation set of 10% for Model Selection and report performance on the remaining 20% held out test set.

In [ ]:
n = len(x)

tr_size = int(n*0.70)
vl_size = int(n*0.10)
te_size = int(n*0.20)

x_train, x_test, x_val = x[:tr_size], x[-te_size:], x[tr_size:tr_size+vl_size]
t_train, t_test, t_val = t[:tr_size], t[-te_size:], t[tr_size:tr_size+vl_size]
e_train, e_test, e_val = e[:tr_size], e[-te_size:], e[tr_size:tr_size+vl_size]

### Setting the parameter grid

Lets set up the parameter grid to tune hyper-parameters. We will tune the number of underlying survival distributions, 
($K$), the distribution choices (Log-Normal or Weibull), the learning rate for the Adam optimizer between $1\times10^{-3}$ and $1\times10^{-4}$ and the number of hidden layers between $0, 1$ and $2$.

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
param_grid = {'k' : [3, 4, 6],
              'distribution' : ['LogNormal', 'Weibull'],
              'learning_rate' : [ 1e-4, 1e-3],
              'layers' : [ [], [100], [100, 100] ]
             }
params = ParameterGrid(param_grid)

### Model Training and Selection

In [ ]:
from auton_survival.models.dsm import DeepSurvivalMachines

In [ ]:
models = []
for param in params:
    model = DeepSurvivalMachines(k = param['k'],
                                 distribution = param['distribution'],
                                 layers = param['layers'])
    # The fit method is called to train the model
    model.fit(x_train, t_train, e_train, iters = 100, learning_rate = param['learning_rate'])
    models.append([[model.compute_nll(x_val, t_val, e_val), model]])
best_model = min(models)
model = best_model[0][1]

  9%|▉         | 9/100 [00:02<00:22,  4.05it/s]


### Inference

In [ ]:
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)

### Evaluation

We evaluate the performance of DSM in its discriminative ability (Time Dependent Concordance Index and Cumulative Dynamic AUC) as well as Brier Score.

In [ ]:
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

In [ ]:
cis = []
brs = []

et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', float)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', float)])
et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
                 dtype = [('e', bool), ('t', float)])

for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
brs.append(brier_score(et_train, et_test, out_survival, times)[1])
roc_auc = []
for i, _ in enumerate(times):
    roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
for horizon in enumerate(horizons):
    print(f"For {horizon[1]} quantile,")
    print("TD Concordance Index:", cis[horizon[0]])
    print("Brier Score:", brs[0][horizon[0]])
    print("ROC AUC ", roc_auc[horizon[0]][0], "\n")

For 0.25 quantile,
TD Concordance Index: 0.7555853798907455
Brier Score: 0.1113088360231026
ROC AUC  0.7649158542616377 

For 0.5 quantile,
TD Concordance Index: 0.699380756108624
Brier Score: 0.18341050843284837
ROC AUC  0.718643804635684 

For 0.75 quantile,
TD Concordance Index: 0.6531435936608161
Brier Score: 0.22063558940580705
ROC AUC  0.7093596655964137 



In [ ]:
from auton_survival.models.dsm import DeepSurvivalMachines
from auton_survival.models.dsm import datasets


# load the SUPPORT dataset.
x, t, e = datasets.load_dataset('SUPPORT')
# instantiate a DeepSurvivalMachines model.
model = DeepSurvivalMachines()
 # fit the model to the dataset.
model.fit(x, t, e)
 # estimate the predicted risks at the time
model.predict_risk(x, 10)

100%|██████████| 1/1 [00:00<00:00,  5.11it/s]


array([[0.1513105 ],
       [0.18146217],
       [0.17543338],
       ...,
       [0.21745589],
       [0.20384838],
       [0.16535427]])

In [ ]:
from auton_survival.models.dsm import DeepSurvivalMachines
model = DeepSurvivalMachines()
model.fit(x, t, e)

100%|██████████| 1/1 [00:00<00:00,  6.01it/s]


https://autonlab.org/auton-survival/models/dsm/index.html#example-usage

from auton_survival.models.dsm.datasets 
from auton_survival.models.dsm import DeepSurvivalMachines
from auton_survival.models.dsm import datasets


# load the SUPPORT dataset.
 x, t, e = datasets.load_dataset('SUPPORT')
# instantiate a DeepSurvivalMachines model.
 model = DeepSurvivalMachines()
 # fit the model to the dataset.
model.fit(x, t, e)
 # estimate the predicted risks at the time
model.predict_risk(x, 10)